In [1]:
import pandas as pd
import numpy as np
!pip install category_encoders

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Read dataset 
df = pd.read_csv('kidney_disease.csv')

In [3]:
# Describe dataset
df

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,47,6700,4.9,no,no,no,good,no,no,notckd
396,396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,54,7800,6.2,no,no,no,good,no,no,notckd
397,397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,49,6600,5.4,no,no,no,good,no,no,notckd
398,398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,51,7200,5.9,no,no,no,good,no,no,notckd


In [71]:
# Separating target from data
X = df.iloc[:,:-1]
y= df.iloc[:, -1]

In [72]:
def normalize(x):
    """
    Function for remove nondesired characters from data
    """
    if type(x) == str:
        x = ''.join(i for i in x.split() if i.isalnum())
        if not x:
            x = np.nan
    return x

In [52]:
# Making maping of columns
maping_y = {'notckd': 0, 'ckd': 1}

In [73]:
# Mapping and cleaning data
y = y.apply(lambda x: maping_y[normalize(x)])
for column in X.columns:
    X[column].apply(lambda x: normalize(x))

In [74]:
# Normalize 
for column in X.columns:
    array_df = X[column]
    new_array = array_df.apply(normalize)
    X[column] = new_array

In [75]:
X.iloc[0, :]

id                0
age            48.0
bp             80.0
sg             1.02
al              1.0
su              0.0
rbc             NaN
pc           normal
pcc      notpresent
ba       notpresent
bgr           121.0
bu             36.0
sc              1.2
sod             NaN
pot             NaN
hemo           15.4
pcv              44
wc             7800
rc              NaN
htn             yes
dm              yes
cad              no
appet          good
pe               no
ane              no
Name: 0, dtype: object

In [76]:
categorical_columns = ['rbc', 'pc', 'ba', 'pcc', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane', 'wc']

In [77]:
# Encoding X data
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
encoder=ce.OneHotEncoder(cols=categorical_columns,handle_unknown='return_nan',return_df=True,use_cat_names=True)
X_encoded = encoder.fit_transform(X)

In [96]:
from sklearn.impute import SimpleImputer
si_mean = SimpleImputer(strategy='mean')

In [97]:
si_mean.fit(X_encoded)

SimpleImputer()

In [98]:
si_mean_transformed_data = si_mean.transform(X_encoded)

In [99]:
si_mean_transformed_data

array([[  0.,  48.,  80., ...,   1.,   0.,   0.],
       [  1.,   7.,  50., ...,   1.,   0.,   0.],
       [  2.,  62.,  80., ...,   0.,   1.,   0.],
       ...,
       [397.,  12.,  80., ...,   1.,   0.,   0.],
       [398.,  17.,  60., ...,   1.,   0.,   0.],
       [399.,  58.,  80., ...,   1.,   0.,   0.]])

In [100]:
# Split data in test and train part
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(si_mean_transformed_data, y.values)

In [101]:
# Import LogisticRegression AND LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

In [102]:
# Initializing Models 
logreg = LogisticRegression(max_iter=5000)
linreg = LinearRegression()

In [103]:
linreg.fit(X_train, y_train)

LinearRegression()

In [104]:
y_pred = linreg.predict(X_test)

In [105]:
y_pred = y_pred.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [106]:
linreg.score(X_test, y_test)

-41385655354482.21